In [1]:
# import libraries

import matplotlib.pylab as plt
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix
import pandas as pd
import numpy as np
from numpy import array
from PIL import Image
import random
from skimage import exposure
import os
import glob
from joblib import Parallel, delayed
from tqdm import tqdm
import cv2

In [2]:
os.chdir('/zfs/analysis/paper/')

In [6]:
for file in glob.glob("input/gem/striatum/*.gem.gz"):
    print(file.split("/")[-1])

FP200000437TR_C1.bin1.gem.gz
FP200000437TR_D1.bin1.gem.gz
FP200000514TL_D5.bin1.gem.gz
FP200000514TL_E3.bin1.gem.gz
FP200000514TL_D3.bin1.gem.gz
FP200000514TL_F3.bin1.gem.gz
FP200000514TL_C3.bin1.gem.gz
FP200000514TL_D2.bin1.gem.gz


In [ ]:
df = pd.read_csv(file, header = 0, sep = '\t', index_col=['geneID'])

df

In [13]:
description = df.groupby(['geneID'])['MIDCounts'].describe()

In [23]:
df.shape[0]/100000

1160.46184

In [36]:
description[description['count'] > 1000].sort_values(by = ['count'], ascending=False).iloc[0:30]

,count,mean,std,min,25%,50%,75%,max
geneID,,,,,,,,
Gm42418,6730543.0,1.474097,0.774047,1.0,1.0,1.0,2.0,47.0
mt-Nd4,1029399.0,1.470321,0.792459,1.0,1.0,1.0,2.0,9.0
mt-Nd1,942873.0,1.432789,0.756350,1.0,1.0,1.0,2.0,11.0
mt-Cytb,751942.0,1.361733,0.682815,1.0,1.0,1.0,2.0,7.0
Fth1,545179.0,1.676407,0.940390,1.0,1.0,1.0,2.0,10.0
Tmsb4x,525844.0,1.568532,0.842018,1.0,1.0,1.0,2.0,8.0
Bc1,471144.0,1.022386,0.151606,1.0,1.0,1.0,1.0,4.0
mt-Co1,421250.0,1.351001,0.676375,1.0,1.0,1.0,2.0,7.0
Ppm1e,401311.0,1.045145,0.262245,1.0,1.0,1.0,1.0,11.0


In [7]:
def create_full_image(gem, output_name, genes):
    
    sample_name = gem.split(".")[0].split("/")[-1]
    
    ################ spot intensity
    output_file_name = output_name + '_' + sample_name + ".png"

    df = pd.read_csv(gem, header = 0, sep = "\t", index_col=['geneID'])
    
    # subset to genes
    df_subset = df.loc[genes]

    # aggregate signal per spot
    spot_intensity = df_subset[["x", "y", "MIDCounts"]].groupby(["x", "y"]).agg('sum').reset_index()

    # clip the top 1%
    p99 = np.percentile(spot_intensity['MIDCounts'], 99)
    spot_intensity['MIDCounts'] = np.clip(spot_intensity['MIDCounts'], 0, p99)

    # rescale to 0-1
    spot_intensity.MIDCounts = spot_intensity.MIDCounts / spot_intensity.MIDCounts.max()

    # make a sparse matrix
    csr_spot_intensity = csr_matrix((spot_intensity.MIDCounts, (spot_intensity.x, spot_intensity.y)))

    # make a dense array
    spot_intensity_arr = csr_spot_intensity.A

    # save the image
    im_255 = spot_intensity_arr * 255
    im_255 = im_255.astype("uint8")
    im = Image.fromarray(im_255, mode='L')
    im.save(os.path.join("input/spot_intensity", output_file_name))
    

In [46]:
create_full_image(file, "test", ['Gm42418', 'mt-Nd4', 'mt-Nd1', 'mt-Cytb', 'Fth1'])

In [11]:
def create_full_image(gem):
    
    sample_name = gem.split(".")[0].split("/")[-1]
    
    ################ spot intensity
    output_spot_intensity_file_name = "spot_intensity_" + sample_name + ".png"

    df = pd.read_csv(gem, header = 0, sep = "\t")

    # aggregate signal per spot
    spot_intensity = df[["x", "y", "MIDCounts"]].groupby(["x", "y"]).agg('sum').reset_index()

    # clip the top 1%
    p99 = np.percentile(spot_intensity['MIDCounts'], 99)
    spot_intensity['MIDCounts'] = np.clip(spot_intensity['MIDCounts'], 0, p99)

    # rescale to 0-1
    spot_intensity.MIDCounts = spot_intensity.MIDCounts / spot_intensity.MIDCounts.max()

    # make a sparse matrix
    csr_spot_intensity = csr_matrix((spot_intensity.MIDCounts, (spot_intensity.x, spot_intensity.y)))

    # make a dense array
    spot_intensity_arr = csr_spot_intensity.A

    # save the image
    im_255 = spot_intensity_arr * 255
    im_255 = im_255.astype("uint8")
    im = Image.fromarray(im_255, mode='L')
    im.save(os.path.join("input/spot_intensity/striatum/", output_spot_intensity_file_name))
    
#     ####################### gm42418
#     output_gm42418_file_name = "gm42418_" + sample_name + ".png"
    
#     Gm42418 = df.copy()
#     Gm42418 = Gm42418.loc[Gm42418['geneID'] == "Gm42418"]
    
#     Gm42418['MIDCounts'] = np.clip(Gm42418['MIDCounts'], 0, np.percentile(Gm42418['MIDCounts'], 98))

#     # rescale to 0-1
#     Gm42418.MIDCounts = Gm42418.MIDCounts / Gm42418.MIDCounts.max()

#     csr_Gm42418 = csr_matrix((Gm42418.MIDCounts, (Gm42418.x, Gm42418.y)))
#     Gm42418_arr = csr_Gm42418.A

#     im_255 = Gm42418_arr * 255
#     im_255 = im_255.astype("uint8")
#     im = Image.fromarray(im_255, mode='L')
#     im.save(os.path.join("input/spot_intensity/striatum/", output_gm42418_file_name))
    

In [12]:
for i in glob.glob("input/gem/striatum/*"):
    print(i)

input/gem/striatum/FP200000437TR_C1.bin1.gem.gz
input/gem/striatum/FP200000437TR_D1.bin1.gem.gz
input/gem/striatum/FP200000514TL_D5.bin1.gem.gz
input/gem/striatum/FP200000514TL_E3.bin1.gem.gz
input/gem/striatum/FP200000514TL_D3.bin1.gem.gz
input/gem/striatum/FP200000514TL_F3.bin1.gem.gz
input/gem/striatum/FP200000514TL_C3.bin1.gem.gz
input/gem/striatum/FP200000514TL_D2.bin1.gem.gz


In [20]:
# os.makedirs("output/full_slice_images")

Parallel(n_jobs=8)(delayed(create_full_image)(gem) for gem in tqdm(glob.glob("input/gem/striatum/*")))


100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 280.44it/s]


[None, None, None, None, None, None, None, None]